## Infos ( à supprimer ensuite)

## Variables à ajouter issues des recherches biblio ( à supprimer ensuite)

Variable(s): 

- Population des pays représentés 
- Variable indiquant si le pays est communiste ou pas 
- Variable indiquant si le pays est stable ou pas 
- Variable indiquant si le pays est émergeant ou pas ( otherwise pauvre , riche à peaufiner par la suite) 
- Variable indiquant si le pays a la culture du sport ou pas. 
- Variable indiquant le PIB par habitant 
- Variable indiquant les financements alloués aux sports (olympiques ou pas)


Il faut créer le train et le test avec des données 2024 , il faut les construire , il faut aussi savoir traiter le fait que cette fois ci la Russie ne sera pas disponible .
Faire indice : pays perturbé 1 avec Russie par exemple ou bien on enlève la Russie  (contexte géopolitique simple, on tah les années où j'ai eu un pb )

But c'est de prédire le nombre de médailles pour chaque pays , indicateur simple 
EXTRA POUR AVOIR 20 : Attribuer un poid aux médailles -> pour potentiellement savoir le nb d'or , argent , bronze

Attention au COVID 19 , influence public , web scrapping sur 30 ans de tous les pays qui ont organisé les jeux 
Attention aux pays pas autorisés à concourir 
Pays en guerre qui envoient moins d'athlètes 
Ukraine 
Palestine 
Iles caiman -> doivent être rattachées à un pays 

JO hiver 
JO été Corrélation 
Apparition des femmes dans les JO ( échantillonnage ) 

Culture du sport : Comment la créer ? 
Budget alloué aux sports (investissement , infrastructures ... ) 
Nb de professionnels 
Nb de licencés 
Nb de sports olympiques disponibles à l'école 

Variable : Pays Communiste ou pas 

Croissance pays économique 

Athlète à partir de 25 ans il n'a plus le niveau : on commence à partir de 2000 , JO Sydney 

Indice dvp des pays à recevoir 

Variable pays organisateur 

Sport qui ont des gens avec un âge bien définie (exemple skate Japonais c'est que des jeunes <= 18 ans)

Dataset coupes mondes de discipline  
    
Variable Note selon l'âge 

Athlète qui sont comptés comme participant à deux pays 

Variable : religion majoritaire pays 

Espérance de vie par pays 
Variable pays dvp ou pas 


Recupérer données JO 2018 et 2021

Variable Poids par Médailles : échelloner les JO de 1 à 10 pour 
Variable Poids par année : 

# Introduction  (Business Understanding) 

## Détermination objectifs métier, analytique du projet 

## Identifier les contraintes, limites et risque du projet

## Chargement des packages si besoin 

In [118]:
%load_ext ipython_memory_usage 
!pip install ydata_profiling

from ydata_profiling import ProfileReport

import pandas as pd
import numpy as np
import ipython_memory_usage.ipython_memory_usage as imu
from describe_csv import *  # charge le fichier describe_csv.py par chunk de 10 000 (10**4)

import math
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.pyplot import *
import scipy as sp
import scipy.stats
from pivottablejs import pivot_ui
import seaborn as sns

from sklearn.impute import SimpleImputer
from sklearn.impute import KNNImputer
from sklearn import metrics
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import train_test_split

from sklearn.pipeline import make_pipeline, make_union
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer, make_column_transformer, make_column_selector
from sklearn.preprocessing import OneHotEncoder, StandardScaler, RobustScaler
from sklearn.preprocessing import FunctionTransformer
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier


from bs4 import BeautifulSoup
import requests
import re
import datetime

imu.start_watching_memory() 

The ipython_memory_usage extension is already loaded. To reload it, use:
  %reload_ext ipython_memory_usage
In [118] used -6.7 MiB RAM in 2.24s (system mean cpu 43%, single max cpu 100%), peaked 6.7 MiB above final usage, current RAM usage now 1344.8 MiB


# CODE  

## Bases de données disponibles 

In [119]:
df_athlete=pd.read_csv('data/athlete_events.csv')
df_region=pd.read_csv('data/noc_regions.csv')
df_dictionnary=pd.read_csv('data/worldcities.csv', sep=';')

In [119] used 34.5 MiB RAM in 0.55s (system mean cpu 31%, single max cpu 92%), peaked 0.0 MiB above final usage, current RAM usage now 1379.2 MiB


#### Pays organisateurs

In [120]:
# TODO: Changer le nom des variables
url='https://www.topendsports.com/events/summer/hosts/list.htm'
response=requests.get(url)
url_2='https://en.wikipedia.org/wiki/List_of_Olympic_Games_host_cities'

response_2=requests.get(url_2)

soup_2=BeautifulSoup(response_2.text,'html.parser')

In [120] used 1.7 MiB RAM in 1.40s (system mean cpu 26%, single max cpu 100%), peaked 0.0 MiB above final usage, current RAM usage now 1380.9 MiB


In [121]:
# TODO: Changer le nom des variables

# Récupération des éléments de la page
L_pays=[]
L_annees=[]
elements=soup_2.findAll('span',class_='datasortkey')

for element in elements:
    pays=element.find('a').text
    L_pays.append(pays)

L_pays=L_pays[42:58]

In [121] used -0.0 MiB RAM in 0.12s (system mean cpu 27%, single max cpu 62%), peaked 0.0 MiB above final usage, current RAM usage now 1380.9 MiB


In [122]:
# Itération des années
L_annees_2=[2000 + i  for i in range(0,34,2)]
L_annees_2.pop(-2)

L_annees_2

[2000,
 2002,
 2004,
 2006,
 2008,
 2010,
 2012,
 2014,
 2016,
 2018,
 2020,
 2022,
 2024,
 2026,
 2028,
 2032]

In [122] used 0.1 MiB RAM in 0.10s (system mean cpu 23%, single max cpu 60%), peaked 0.0 MiB above final usage, current RAM usage now 1381.0 MiB


In [123]:
# On lie les pays et les années

L_pays_annee=[]
for i in range(len(L_pays)):
    L_pays_annee.append((L_pays[i],L_annees_2[i]))

df_hote=pd.DataFrame(L_pays_annee,columns=['Host_country','Year'])

df_hote

,Host_country,Year
0,Australia,2000
1,United States,2002
2,Greece,2004
3,Italy,2006
4,China,2008
5,Canada,2010
6,United Kingdom,2012
7,Russia,2014
8,Brazil,2016
9,South Korea,2018


In [123] used 0.0 MiB RAM in 0.11s (system mean cpu 29%, single max cpu 68%), peaked 0.0 MiB above final usage, current RAM usage now 1381.0 MiB


#### Pays avec un passé communiste

In [124]:
url_soviet='https://en.wikipedia.org/wiki/Post-Soviet_states'
response_soviet=requests.get(url_soviet)
soup_soviet=BeautifulSoup(response_soviet.text,'html.parser')

L_pays_soviet=[]

pays_soviet=soup_soviet.findAll('b')
#print(pays_soviet)
for pays in pays_soviet:
    b=pays.find('a')
    if b is not None:
        L_pays_soviet.append(b.text)
L_pays_soviet=L_pays_soviet[:15]

complete_urss_regime=lambda x : 1 if x in L_pays_soviet else 0

In [124] used 1.0 MiB RAM in 0.55s (system mean cpu 39%, single max cpu 100%), peaked 0.0 MiB above final usage, current RAM usage now 1382.1 MiB


#### Scrap population et GDP

In [125]:
url_pop_gpa='https://www.worldometers.info/gdp/gdp-by-country/'
response_pop_gpa=requests.get(url_pop_gpa)
soup_pop_gpa=BeautifulSoup(response_pop_gpa.text,'html.parser')

L_pays=[]

all_countries=soup_pop_gpa.findAll('td')

for country in all_countries:
    a=country.find('a')
    if a is not None:
        L_pays.append(a.text)
len(L_pays)

177

In [125] used -0.0 MiB RAM in 0.62s (system mean cpu 25%, single max cpu 80%), peaked 0.0 MiB above final usage, current RAM usage now 1382.1 MiB


In [126]:
all_info=soup_pop_gpa.findAll('td',attrs={'style':'font-weight: bold; text-align:right;'})

numbers=[]
for i,info in enumerate(all_info):
    b=re.findall(r'\d+\b', info.text)
    b=''.join(b)
    numbers.append(b)
   
population=[]
gpd_per_habitant=[]
PIB_liste=[]
for i in range(0,len(numbers),6):
    PIB_liste.append(int(numbers[i]))
for i in range(3,len(numbers),6):
    population.append(int(numbers[i]))
for i in range(4,len(numbers),6):
    gpd_per_habitant.append(int(numbers[i]))

print(len(PIB_liste))
PIB_liste

177


[25462700000000,
 17963200000000,
 4231140000000,
 4072190000000,
 3385090000000,
 3070670000000,
 2782910000000,
 2240420000000,
 2139840000000,
 2010430000000,
 1920100000000,
 1675420000000,
 1665250000000,
 1414190000000,
 1397510000000,
 1319100000000,
 1108150000000,
 991115000000,
 905988000000,
 807706000000,
 688177000000,
 632770000000,
 585939000000,
 579267000000,
 578604000000,
 529245000000,
 522033000000,
 507535000000,
 495341000000,
 477386000000,
 476748000000,
 471400000000,
 466789000000,
 460201000000,
 408802000000,
 406306000000,
 405870000000,
 404284000000,
 395404000000,
 388544000000,
 376533000000,
 359839000000,
 343939000000,
 301262000000,
 301025000000,
 290924000000,
 280826000000,
 264182000000,
 251945000000,
 247234000000,
 242632000000,
 237296000000,
 220623000000,
 219066000000,
 191913000000,
 184558000000,
 178789000000,
 160503000000,
 134182000000,
 126783000000,
 115469000000,
 115049000000,
 114667000000,
 113642000000,
 113420000000,
 10671

In [126] used 0.0 MiB RAM in 0.12s (system mean cpu 25%, single max cpu 47%), peaked 0.0 MiB above final usage, current RAM usage now 1382.1 MiB


In [127]:
# On replace les pays qui n'ont pas le même nom
df_pop_gpd=pd.DataFrame({'Country':L_pays,'PIB':PIB_liste,'Population':population,'PIB/habitant':gpd_per_habitant})

df_pop_gpd['Country']=df_pop_gpd['Country'].replace('Trinidad and Tobago','Trinidad')
df_pop_gpd['Country']=df_pop_gpd['Country'].replace('State of Palestine','Palestine, Occupied Territories')
df_pop_gpd['Country']=df_pop_gpd['Country'].replace('Singapore','Singapour')
df_pop_gpd['Country']=df_pop_gpd['Country'].replace('Czech Republic (Czechia)','Czech Republic')
df_pop_gpd['Country']=df_pop_gpd['Country'].replace("Côte d'Ivoire" ,"Cote d'Ivoire")

df_pop_gpd

,Country,PIB,Population,PIB/habitant
0,United States,25462700000000,338289857,75269
1,China,17963200000000,1425887337,12598
2,Japan,4231140000000,123951692,34135
3,Germany,4072190000000,83369843,48845
4,India,3385090000000,1417173173,2389
...,...,...,...,...
172,Sao Tome & Principe,546680342,227380,2404
173,Micronesia,427094119,539013,792
174,Marshall Islands,279667900,41569,6728
175,Kiribati,223352943,131232,1702


In [127] used 0.1 MiB RAM in 0.11s (system mean cpu 25%, single max cpu 62%), peaked 0.0 MiB above final usage, current RAM usage now 1382.1 MiB


In [128]:
# Pays qui manquaient sur le site scrappé, données récupérées manuellement
df_pop_gpd=pd.concat([df_pop_gpd,pd.DataFrame([['Taiwan',762670000000,23923276,44460],
                                               ['Cook Islands',220000000,17044,13558],
                                               ['Eritrea',1960000000,3684000,1067],
                                               ['Kosovo',9410000000,1760000,5340],
                                               ['Lebanon',23130000000,5425542,4136],
                                               ['Netherlands Antilles',3810000000,27148,11400],
                                               ['South Sudan',12000000000,11000000,467], 
                                               ['American Samoa',871000000,43914,19673],
                                              ['Aruba',3545000000,106445,333000],
                                              ['Bermuda',7645000000,63532,118774],
                                               ['British Virgin Islands',1751000000,31707,34246],
                                               ['Virgin Islands',4440000000,105413,41974],
                                               ['Cayman Islands',6845000000,69310,99624],
                                               ['Cuba',633400000000,11179471,56495],
                                               ['Guam',6123000000,173825,35904],
                                               ['North Korea',48000000000,26070000,33719],
                                               ['Liechtenstein',7710000000,39039,197504],
                                               ['Monaco',8784000000,39050,240862],
                                               ['Puerto Rico',113400000000,3260314,35208],
                                               ['San Marino',1855000000,33642,53000],
                                               ['Syria',8970000000,22130000,420],
                                               ['Venezuela',92000000000,29254044,2624],
                                               ['Virgin Islands*',4444000000,105413,41976],
                                               ['Afghanistan',14583135237,42239000,363.73],
                                               ['Bhutan',2539551327,782455,3560],
                                               ['Brunei',16760000000,449000,37152],
                                               ['Burma',62260000000,54180000,1149],
                                               ['Cape Verde',2227000000,593000,3754],
                                               ['Djibouti',3515000000,1121000,3136],
                                               ["East Timor (Timor-Leste)",3205000000,1341000,2389],
                                               ['Macedonia',13560000000,2058000,6591],
                                               ['Nauru',151600000,12668,11970],
                                               ['Palau',232000000,18055,12921],
                                               ['Saint Kitts and Nevis',965000000,47000,20262],
                                               ['Saint Vincent and the Grenadines',948000000,9123,9125],
                                               ['Sao Tome and Principe',542000000,227380,2386],
                                               ['Somalia',10042000000,17000000,592],
                                               ['Swaziland',4791000000,1202000,3986],
                                               ['Tonga',469000000,106858,4425],
                                               ['Turkmenistan',56540000000,6431000,8792],
                                               ['Yemen',21160000000,33700000,650]
                                               ],
                                             columns=['Country','PIB','Population','PIB/habitant'])],ignore_index=True)

In [128] used 0.0 MiB RAM in 0.11s (system mean cpu 25%, single max cpu 60%), peaked 0.0 MiB above final usage, current RAM usage now 1382.1 MiB


In [129]:
## Dictionnaire pour remplir les valeurs NaN de Pop et PIB sans changer les valeurs de pop et PIB per habitant précédentes
dictio_pop_gpd=dict(zip(df_pop_gpd['Country'],df_pop_gpd['PIB']))

dictio_pop_gpd

{'United States': 25462700000000,
 'China': 17963200000000,
 'Japan': 4231140000000,
 'Germany': 4072190000000,
 'India': 3385090000000,
 'United Kingdom': 3070670000000,
 'France': 2782910000000,
 'Russia': 2240420000000,
 'Canada': 2139840000000,
 'Italy': 2010430000000,
 'Brazil': 1920100000000,
 'Australia': 1675420000000,
 'South Korea': 1665250000000,
 'Mexico': 1414190000000,
 'Spain': 1397510000000,
 'Indonesia': 1319100000000,
 'Saudi Arabia': 1108150000000,
 'Netherlands': 991115000000,
 'Turkey': 905988000000,
 'Switzerland': 807706000000,
 'Poland': 688177000000,
 'Argentina': 632770000000,
 'Sweden': 585939000000,
 'Norway': 579267000000,
 'Belgium': 578604000000,
 'Ireland': 529245000000,
 'Israel': 522033000000,
 'United Arab Emirates': 507535000000,
 'Thailand': 495341000000,
 'Nigeria': 477386000000,
 'Egypt': 476748000000,
 'Austria': 471400000000,
 'Singapour': 466789000000,
 'Bangladesh': 460201000000,
 'Vietnam': 408802000000,
 'Malaysia': 406306000000,
 'South Afr

In [129] used 0.0 MiB RAM in 0.11s (system mean cpu 30%, single max cpu 71%), peaked 0.0 MiB above final usage, current RAM usage now 1382.1 MiB


#### JO de Tokyo

Pour les jeux de 2020 (été, Tokyo), nous avons trouvé le dataset suivant : https://www.kaggle.com/datasets/piterfm/tokyo-2020-olympics. Après téléchargement des différents datasets, nous le plaçons dans `./data/additionnal/tokyo2020`

In [130]:
# Chargement des données
df_tokyo_athletes = pd.read_csv('data/additionnal/tokyo2020/athletes.csv')
df_tokyo_medals_total = pd.read_csv('data/additionnal/tokyo2020/medals_total.csv')
df_tokyo_medals = pd.read_csv('data/additionnal/tokyo2020/medals.csv')

In [130] used 3.8 MiB RAM in 0.14s (system mean cpu 35%, single max cpu 67%), peaked 0.0 MiB above final usage, current RAM usage now 1386.0 MiB


In [131]:
# Création d'un dataframe avec les même colonnes que df_athlete
df_complete_tokyo2020 = pd.DataFrame(columns=df_athlete.columns)

# On s'assure qu'il n'y a pas de doublons
df_tokyo_athletes.drop_duplicates(inplace=True)

df_merged_bis = pd.merge(df_tokyo_athletes, df_tokyo_medals, how='left', left_on=['name', 'discipline_code'], right_on=['athlete_name', 'discipline_code'])

In [131] used 0.2 MiB RAM in 0.14s (system mean cpu 23%, single max cpu 80%), peaked 0.0 MiB above final usage, current RAM usage now 1386.2 MiB


In [132]:
athletes_to_be_added = []
# Itération dans df_tokyo_athletes pour ajouter dans athletes_to_be_added, qui sera ensuite ajouté au dataframe df_complete_tokyo2020

for index, row in df_tokyo_athletes.iterrows():
    athlete_name = row['name']
    athlete_sex = str(row['gender'])[0] if type(row['gender'] != float) else np.nan
    athlete_age = (2021 - datetime.date.fromisoformat(str(row['birth_date'])).year) if (type(row['birth_date'] == str) and str(row['birth_date']).lower() != 'nan' ) else np.nan
    athlete_height_in_m = (int(float(row['height_m/ft'].split('/')[0]) *100)) if (type(row['height_m/ft']) != float) else np.nan
    athlete_team = row['country']
    athlete_noc = row['country_code']
    athlete_games = 'Tokyo 2020'
    athlete_year = 2020
    athlete_season = 'Summer'
    athlete_city = 'Tokyo'
    athlete_discipline = row['discipline']
    athletes_to_be_added.append({
        'Name': athlete_name,
        'Sex': athlete_sex, 
        'Age': athlete_age,
        'Height': athlete_height_in_m,
        'Team': athlete_team,
        'NOC': athlete_noc,
        'Games': athlete_games,
        'Year': athlete_year,
        'Season': athlete_season,
        'City': athlete_city,
        'Sport': athlete_discipline,
    })

df_complete_tokyo2020 = df_complete_tokyo2020.from_records(athletes_to_be_added)

In [132] used 0.7 MiB RAM in 0.61s (system mean cpu 24%, single max cpu 100%), peaked 0.0 MiB above final usage, current RAM usage now 1386.9 MiB


In [133]:
# On effectue une jointure droite entre les deux dataframes, de manière à conserver seulement les athlètes avec différentes médailles dans différentes disciplines
df_merged = pd.merge(df_complete_tokyo2020, df_tokyo_medals, left_on=['Name', 'Sport'], right_on=['athlete_name', 'discipline'], how='left')
df_merged.drop(columns=['medal_code', 'medal_date', 'athlete_short_name',
       'athlete_name', 'athlete_sex', 'athlete_link', 'country_code',
       'discipline_code', 'country', 'discipline'], inplace=True)

df_merged.drop_duplicates(inplace=True)

df_merged['medal_type'] = df_merged['medal_type'].apply(lambda x: x.replace('Medal', '').strip() if type(x) != float else x)
df_merged.rename(columns={'medal_type': 'Medal', 'event': 'Event'}, inplace=True)

In [133] used -0.1 MiB RAM in 0.14s (system mean cpu 64%, single max cpu 95%), peaked 0.0 MiB above final usage, current RAM usage now 1386.9 MiB


In [134]:
# On récupère le nom des médaillés seulement, pour les retirer du dataframe final (qui ne contient pas encore les médailles obtenues par les athlètes), avant de fusionner
# le dataframe des médaillés seulement et celui des athlètes sans médailles
medailles_seulement = df_merged.Name.unique()
df_complete_tokyo2020.drop(df_complete_tokyo2020[df_complete_tokyo2020['Name'].isin(medailles_seulement)].index, inplace=True)
df_complete_tokyo2020 = pd.concat([df_complete_tokyo2020, df_merged])
df_complete_tokyo2020

,Name,Sex,Age,Height,Team,NOC,Games,Year,Season,City,Sport,Medal,Event
0,AALERUD Katrine,F,27.0,NaN,Norway,NOR,Tokyo 2020,2020,Summer,Tokyo,Cycling Road,NaN,NaN
1,ABAD Nestor,M,28.0,165.0,Spain,ESP,Tokyo 2020,2020,Summer,Tokyo,Artistic Gymnastics,NaN,NaN
2,ABAGNALE Giovanni,M,26.0,198.0,Italy,ITA,Tokyo 2020,2020,Summer,Tokyo,Rowing,NaN,NaN
3,ABALDE Alberto,M,26.0,200.0,Spain,ESP,Tokyo 2020,2020,Summer,Tokyo,Basketball,NaN,NaN
4,ABALDE Tamara,F,32.0,192.0,Spain,ESP,Tokyo 2020,2020,Summer,Tokyo,Basketball,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11876,ZWICKER Martin Detlef,M,34.0,176.0,Germany,GER,Tokyo 2020,2020,Summer,Tokyo,Hockey,NaN,NaN
11877,ZWOLINSKA Klaudia,F,23.0,NaN,Poland,POL,Tokyo 2020,2020,Summer,Tokyo,Canoe Slalom,NaN,NaN
11878,ZYKOVA Yulia,F,26.0,NaN,ROC,ROC,Tokyo 2020,2020,Summer,Tokyo,Shooting,Silver,50m Rifle 3 Positions Women
11879,ZYUZINA Ekaterina,F,25.0,NaN,ROC,ROC,Tokyo 2020,2020,Summer,Tokyo,Sailing,NaN,NaN


In [134] used 0.0 MiB RAM in 0.12s (system mean cpu 51%, single max cpu 100%), peaked 0.0 MiB above final usage, current RAM usage now 1386.9 MiB


In [135]:
# On remplace les valeurs 'n' en NaN pour le sexe
df_complete_tokyo2020['Sex'] = df_complete_tokyo2020['Sex'].apply(lambda x: np.NAN if x == 'n' else x)

In [135] used 0.0 MiB RAM in 0.11s (system mean cpu 45%, single max cpu 100%), peaked 0.0 MiB above final usage, current RAM usage now 1386.9 MiB


In [136]:
# On merge avec df_athlete
df_athlete = pd.concat([df_athlete,df_complete_tokyo2020 ])

In [136] used 8.6 MiB RAM in 0.15s (system mean cpu 47%, single max cpu 100%), peaked 0.0 MiB above final usage, current RAM usage now 1395.5 MiB


## Optimisation des datasets
   Dans un souci d'optimisation de l'espace mémoire occupé, nous allons faire en sorte d'optimiser les différents dataframe que nous utilisons. 
Pour mener à bien ceci, nous allons des fonctions contenues dans `describe_csv.py`, en les améliorant pour automatiser le processus.

In [137]:
#### Code Nathan à ajouter
# Jusqu'à maintenant, nous utilisons le dataframe df_athlete. Nous allons définir différentes fonctions pour optimiser les types et ainsi optimiser l'espace mémoire occupé

from pandas.api.types import is_numeric_dtype


def define_and_use_optimal_type(df:pd.DataFrame):
    for column in df.columns:
        if(is_numeric_dtype(df[column]) == True):
            # Set optimal type else do nothing
            optimalype = optimal_type(df[column], df[column].min(), df[column].max())
            df[column] = df[column].astype(optimalype)
    return df

In [137] used 0.0 MiB RAM in 0.11s (system mean cpu 48%, single max cpu 100%), peaked 0.0 MiB above final usage, current RAM usage now 1395.6 MiB


## Nettoyage

In [138]:
## Ajouter le code FLO

In [138] used -0.0 MiB RAM in 0.10s (system mean cpu 39%, single max cpu 86%), peaked 0.0 MiB above final usage, current RAM usage now 1395.5 MiB


### Dataframe dictionnary

In [139]:
df_dictionnary=pd.read_csv('data/dictionary.csv')

df_dictionnary=df_dictionnary.rename(columns={'GDP per Capita':'PIB/habitant','Code':'NOC'})

df_dictionnary.isna().sum()

Country          0
NOC              0
Population       5
PIB/habitant    25
dtype: int64

In [139] used -0.0 MiB RAM in 0.11s (system mean cpu 47%, single max cpu 84%), peaked 0.0 MiB above final usage, current RAM usage now 1395.5 MiB


### Dataframe région

In [140]:
df_region=df_region.rename(columns={'region':'Country'})

In [140] used 0.0 MiB RAM in 0.10s (system mean cpu 46%, single max cpu 89%), peaked 0.0 MiB above final usage, current RAM usage now 1395.5 MiB


Ajout des îles Tuvalu

In [141]:
df_region[df_region['Country'].isnull()]

,NOC,Country,notes
168,ROT,NaN,Refugee Olympic Team
208,TUV,NaN,Tuvalu
213,UNK,NaN,Unknown


In [141] used -0.0 MiB RAM in 0.11s (system mean cpu 38%, single max cpu 75%), peaked 0.0 MiB above final usage, current RAM usage now 1395.5 MiB


In [142]:
df_region=df_region.query(" NOC != 'ROT' and NOC != 'UNK' ")

df_region['Country']=df_region['Country'].fillna('Tuvalu')

In [142] used 0.1 MiB RAM in 0.11s (system mean cpu 59%, single max cpu 89%), peaked 0.0 MiB above final usage, current RAM usage now 1395.6 MiB


In [143]:
df_region

,NOC,Country,notes
0,AFG,Afghanistan,NaN
1,AHO,Curacao,Netherlands Antilles
2,ALB,Albania,NaN
3,ALG,Algeria,NaN
4,AND,Andorra,NaN
...,...,...,...
225,YEM,Yemen,NaN
226,YMD,Yemen,South Yemen
227,YUG,Serbia,Yugoslavia
228,ZAM,Zambia,NaN


In [143] used -0.0 MiB RAM in 0.11s (system mean cpu 58%, single max cpu 100%), peaked 0.0 MiB above final usage, current RAM usage now 1395.6 MiB


## Merge des dataframes et traitement des valeurs nulles

### Opérations sur les pays

In [144]:
df_region

,NOC,Country,notes
0,AFG,Afghanistan,NaN
1,AHO,Curacao,Netherlands Antilles
2,ALB,Albania,NaN
3,ALG,Algeria,NaN
4,AND,Andorra,NaN
...,...,...,...
225,YEM,Yemen,NaN
226,YMD,Yemen,South Yemen
227,YUG,Serbia,Yugoslavia
228,ZAM,Zambia,NaN


In [144] used 0.0 MiB RAM in 0.11s (system mean cpu 52%, single max cpu 100%), peaked 0.0 MiB above final usage, current RAM usage now 1395.6 MiB


In [145]:
df_complete=pd.merge(pd.merge(df_athlete,df_region,on='NOC',how='left'),df_dictionnary,on='NOC',how='left')

# On prend seulement les JO d'été
df_complete=df_complete[df_complete['Season']=='Summer']
df_complete=df_complete[df_complete['Year'] >= 2000]

In [145] used 68.5 MiB RAM in 0.40s (system mean cpu 63%, single max cpu 100%), peaked 0.0 MiB above final usage, current RAM usage now 1464.1 MiB


In [146]:
# Attribution des pays en fonction des NOC 
df_complete=pd.merge(df_complete,df_region,on='NOC',how='left')
df_complete['Country']=df_complete['Country_y'].fillna(df_complete['Country_x'])
df_complete= df_complete.drop(['Country_x', 'Country_y', 'notes_x','notes_y'], axis=1)

In [146] used 1.0 MiB RAM in 0.16s (system mean cpu 60%, single max cpu 100%), peaked 0.0 MiB above final usage, current RAM usage now 1465.1 MiB


In [147]:
# On retire les NOC (réfugiés)
df_complete=df_complete.query(" NOC !='ROC' ")

In [147] used 0.0 MiB RAM in 0.13s (system mean cpu 53%, single max cpu 100%), peaked 0.0 MiB above final usage, current RAM usage now 1465.1 MiB


In [148]:
# Analyse des valeurs nulles 
df_complete.isna().sum()

# 156 sports où on sait pas ce que c'est .... et des country (26 221) qui ne sont pas attribués non plus 

Name                0
Sex               312
Age               315
Height          14603
Weight          23938
Team                0
NOC                 0
Games               0
Year                0
Season              0
City                0
Sport             312
Event           18563
Medal           76020
Population       3088
PIB/habitant     6076
Country           277
dtype: int64

In [148] used 0.0 MiB RAM in 0.20s (system mean cpu 32%, single max cpu 100%), peaked 0.0 MiB above final usage, current RAM usage now 1465.1 MiB


In [149]:
# Attribution des 
liste_pays=df_region['Country'].tolist()
liste_NOC=df_region['NOC'].tolist()

dictio_NOC_pays=dict(zip(liste_NOC,liste_pays))

dictio_NOC_pays['SGP']='Singapour'
dictio_NOC_pays['LBN']='Liban'

df_complete['Country']=df_complete['Country'].fillna(df_complete['NOC'].map(dictio_NOC_pays))


In [149] used -0.0 MiB RAM in 0.12s (system mean cpu 52%, single max cpu 100%), peaked 0.0 MiB above final usage, current RAM usage now 1465.1 MiB


In [150]:
# Il y a quatre NOC qui ne correspondent à aucun pays dans notre EOR : réfugiés , ROT réfugiés aussi , LBN c'est le Liban (Lebanon en anglais )
df_complete=df_complete.query(" NOC !='ROT' and NOC !='EOR' ")

In [150] used 0.0 MiB RAM in 0.13s (system mean cpu 47%, single max cpu 100%), peaked 0.0 MiB above final usage, current RAM usage now 1465.1 MiB


In [151]:
# On ajoute les pays organisateurs pour chaque ligne scrappée précédemment
df_complete=pd.merge(df_complete,df_hote,on='Year',how='left')

In [151] used 0.0 MiB RAM in 0.13s (system mean cpu 42%, single max cpu 100%), peaked 0.0 MiB above final usage, current RAM usage now 1465.1 MiB


In [152]:
# On ajoute une colonne booléenne (l'athlète participe aux jeux dans son propre pays)
compare_col=lambda x : 1 if x['Country']==x['Host_country'] else 0 

df_complete['Participation_own_country']=df_complete.apply(compare_col,axis=1)
df_complete[df_complete['Participation_own_country']==1]

,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal,Population,PIB/habitant,Country,Host_country,Participation_own_country
38,Giorgos Abalof,M,22.0,178.0,70.0,Greece,GRE,2004 Summer,2004,Summer,Athina,Fencing,"Fencing Men's epee, Individual",NaN,10823732.0,18002.230578,Greece,Greece,1
40,Georgios Abaris,M,22.0,190.0,77.0,Greece,GRE,2004 Summer,2004,Summer,Athina,Football,Football Men's Football,NaN,10823732.0,18002.230578,Greece,Greece,1
49,Georgia Abatzidou,F,35.0,155.0,43.0,Greece,GRE,2004 Summer,2004,Summer,Athina,Athletics,Athletics Women's Marathon,NaN,10823732.0,18002.230578,Greece,Greece,1
156,Ali Abdo,M,19.0,169.0,74.0,Australia,AUS,2000 Summer,2000,Summer,Sydney,Wrestling,"Wrestling Men's Welterweight, Greco-Roman",NaN,23781169.0,56310.962993,Australia,Australia,1
268,Diana Monteiro Abla,F,21.0,175.0,75.0,Brazil,BRA,2016 Summer,2016,Summer,Rio de Janeiro,Water Polo,Water Polo Women's Water Polo,NaN,207847528.0,8538.589975,Brazil,Brazil,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90220,YOSHINO Tatsuki,M,27.0,NaN,NaN,Japan,JPN,Tokyo 2020,2020,Summer,Tokyo,Handball,NaN,NaN,126958472.0,32477.215145,Japan,Japan,1
90221,YOSHIOKA Dai,M,36.0,NaN,NaN,Japan,JPN,Tokyo 2020,2020,Summer,Tokyo,Shooting,NaN,NaN,126958472.0,32477.215145,Japan,Japan,1
90222,YOSHIOKA Miho,F,31.0,177.0,NaN,Japan,JPN,Tokyo 2020,2020,Summer,Tokyo,Sailing,NaN,NaN,126958472.0,32477.215145,Japan,Japan,1
90223,YOSOZUMI Sakura,F,19.0,NaN,NaN,Japan,JPN,Tokyo 2020,2020,Summer,Tokyo,Skateboarding,Women's Park,Gold,126958472.0,32477.215145,Japan,Japan,1


In [152] used 0.0 MiB RAM in 0.65s (system mean cpu 53%, single max cpu 100%), peaked 0.0 MiB above final usage, current RAM usage now 1465.1 MiB


In [153]:
# On récupère les pays communistes scrappés précédemment
df_complete['Soviet_past']=df_complete['Country'].apply(complete_urss_regime)

In [153] used -0.0 MiB RAM in 0.14s (system mean cpu 46%, single max cpu 100%), peaked 0.0 MiB above final usage, current RAM usage now 1465.1 MiB


In [154]:
# On renomme les pays pour pouvoir merge avec les données scrappées 
# Changement de noms pour les pays

df_complete['Country']=df_complete['Country'].replace('Puerto Rico*','Puerto Rico')
df_complete['Country']=df_complete['Country'].replace('Hong Kong*','Hong Kong')
df_complete['Country']=df_complete['Country'].replace('Korea, South','South Korea')
df_complete['Country']=df_complete['Country'].replace('Korea, North','South Korea')
df_complete['Country']=df_complete['Country'].replace('Liban','Lebanon')
df_complete['Country']=df_complete['Country'].replace('Congo, Dem Rep','DR Congo')
df_complete['Country']=df_complete['Country'].replace('Cayman Islands*','Cayman Islands')
df_complete['Country']=df_complete['Country'].replace('Bermuda*','Bermuda')
df_complete['Country']=df_complete['Country'].replace('Virgin Islands*','Virgin Islands')
df_complete['Country']=df_complete['Country'].replace('Aruba*','Aruba')
df_complete['Country']=df_complete['Country'].replace('American Samoa*','American Samoa')
df_complete['Country']=df_complete['Country'].replace('Netherlands Antilles*','Netherlands Antilles')

# URSS 
df_complete.loc[(df_complete['Team'] == 'Soviet Union') & (df_complete['NOC'] == 'URS'), ['Team', 'NOC']] = ['Russia', 'RUS'] # URSS période 1952-1988

# Réfugiés
df_complete.loc[(df_complete['Team'] == 'ROC') & (df_complete['NOC'] == 'ROC'), ['Team', 'NOC']] = ['Germany', 'RUS'] # JO 2020

# GERMANY
df_complete.loc[(df_complete['Team'] == 'East Germany') & (df_complete['NOC'] == 'GDR'), ['Team', 'NOC']] = ['Germany', 'GER']

# CHINA
df_complete.loc[df_complete['Team'] == "People's Republic of China", ['Team', 'NOC']] = ['China', 'CHN']

# UNITED STATES
df_complete.loc[df_complete['Team'] == "United States of America", ['Team', 'NOC']] = ['United States', 'USA']

In [154] used 0.0 MiB RAM in 0.18s (system mean cpu 46%, single max cpu 100%), peaked 0.0 MiB above final usage, current RAM usage now 1465.1 MiB


In [155]:
# Normalisation des pays. Certains sont écrits avec des suffixes (Ex: Brazil-1, Brazil-2)
replacement_dict = {}

old_countries_to_process = df_complete.Team.unique()

for i in old_countries_to_process:
    replacement_dict[i] = re.sub(r'[\d-]', '', i)


final_df = df_complete.replace({"Team": replacement_dict})

In [155] used 0.0 MiB RAM in 0.91s (system mean cpu 48%, single max cpu 100%), peaked 0.0 MiB above final usage, current RAM usage now 1465.2 MiB


In [156]:
df_test_na_pop=df_complete[df_complete['Population'].isna()]

# On merge les données de df_test_na_pop et df_complete
df_test_na_pop=pd.merge(df_test_na_pop,df_pop_gpd,on='Country',how='left')
df_test_na_pop.isna().sum()

Name                            0
Sex                            18
Age                            18
Height                        531
Weight                        844
Team                            0
NOC                             0
Games                           0
Year                            0
Season                          0
City                            0
Sport                          18
Event                         676
Medal                        2599
Population_x                 3018
PIB/habitant_x               3018
Country                         0
Host_country                    0
Participation_own_country       0
Soviet_past                     0
PIB                            17
Population_y                   17
PIB/habitant_y                 17
dtype: int64

In [156] used -0.0 MiB RAM in 0.12s (system mean cpu 48%, single max cpu 100%), peaked 0.0 MiB above final usage, current RAM usage now 1465.1 MiB


In [157]:
# On nettoie les colonnes résiduelles
df_test_na_pop=df_test_na_pop.drop(['Population_x'],axis=1)
df_test_na_pop=df_test_na_pop.drop(['PIB/habitant_x'],axis=1)

# On renomme les colonnes
df_test_na_pop=df_test_na_pop.rename(columns={'Population_y':'Population'})
df_test_na_pop=df_test_na_pop.rename(columns={'PIB/habitant_y':'PIB/habitant'})

# On nettoie les données nulles dans la colonne Population
df_test_na_pop=df_test_na_pop.dropna(subset='Population')
df_complete=df_complete.dropna(subset='Population')

df_complete=pd.concat([df_complete,df_test_na_pop],ignore_index=True)

In [157] used 0.0 MiB RAM in 0.13s (system mean cpu 46%, single max cpu 100%), peaked 0.0 MiB above final usage, current RAM usage now 1465.1 MiB


### Opérations sur le PIB

In [158]:
# Traitement des données nulles
df_test_na_pib=df_complete[df_complete['PIB/habitant'].isna()]
df_complete=df_complete.dropna(subset='PIB/habitant')

# On merge avec les données scrappées
df_test_na_pib=pd.merge(df_test_na_pib,df_pop_gpd,on='Country',how='left')

# On drop les données résiduelles
df_test_na_pib=df_test_na_pib.drop(['Population_x'],axis=1)
df_test_na_pib=df_test_na_pib.drop(['PIB/habitant_x'],axis=1)
df_test_na_pib=df_test_na_pib.drop(['PIB_x'],axis=1)

# On renomme les colonnes merge
df_test_na_pib=df_test_na_pib.rename(columns={'Population_y':'Population'})
df_test_na_pib=df_test_na_pib.rename(columns={'PIB/habitant_y':'PIB/habitant'})
df_test_na_pib=df_test_na_pib.rename(columns={'PIB_y':'PIB'})

df_complete=pd.concat([df_complete,df_test_na_pib],ignore_index=True)

In [158] used 0.0 MiB RAM in 0.15s (system mean cpu 44%, single max cpu 100%), peaked 0.0 MiB above final usage, current RAM usage now 1465.2 MiB


In [159]:
# Attribution du PIB aux pays manquants
df_complete['PIB']=df_complete['PIB'].fillna(df_complete['Country'].map(dictio_pop_gpd))

In [159] used 0.0 MiB RAM in 0.11s (system mean cpu 47%, single max cpu 100%), peaked 0.0 MiB above final usage, current RAM usage now 1465.2 MiB


### Opérations sur les âges

In [160]:
# On attribue les âges manquants en faisant la moyenne des âges par sport
table_sport_age=df_complete.groupby('Sport')['Age'].mean().apply(math.ceil).reset_index()

liste_sport=table_sport_age['Sport'].tolist()
liste_age=table_sport_age['Age'].tolist()

dictio_age_sport=dict(zip(liste_sport,liste_age))

df_complete['Age']=df_complete['Age'].fillna(df_complete['Sport'].map(dictio_age_sport))

In [160] used -0.0 MiB RAM in 0.12s (system mean cpu 35%, single max cpu 71%), peaked 0.0 MiB above final usage, current RAM usage now 1465.2 MiB


In [161]:
# Il y a 156 personnes qui manquent car nous n'avons pas les sports 
df_complete=df_complete.dropna(subset='Age')

In [161] used 2.1 MiB RAM in 0.13s (system mean cpu 38%, single max cpu 82%), peaked 0.0 MiB above final usage, current RAM usage now 1467.2 MiB


### Opérations sur le sexe, le poids et les tailles

In [162]:
df_male = df_complete[df_complete['Sex'] == 'M']
df_female = df_complete[df_complete['Sex'] == 'F']
columns_to_impute = ['Age', 'Height', 'Weight']

In [162] used 0.0 MiB RAM in 0.13s (system mean cpu 23%, single max cpu 70%), peaked 0.0 MiB above final usage, current RAM usage now 1467.2 MiB


In [163]:
# Créer un imputer KNN avec un nombre de voisins à considérer (k)
imputer = KNNImputer(n_neighbors=5)

# Imputer les valeurs manquantes pour les hommes
df_male[columns_to_impute] = imputer.fit_transform(df_male[columns_to_impute])

# Imputer les valeurs manquantes pour les femmes
df_female[columns_to_impute] = imputer.fit_transform(df_female[columns_to_impute])

# Fusionner les DataFrames pour obtenir le DataFrame complet
df_complete = pd.concat([df_male, df_female])

/var/folders/38/1vcxjcxx0z75mdh8whh4pw6r0000gn/T/ipykernel_70604/3237411982.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_male[columns_to_impute] = imputer.fit_transform(df_male[columns_to_impute])


In [163] used -869.2 MiB RAM in 50.79s (system mean cpu 25%, single max cpu 100%), peaked 3005.5 MiB above final usage, current RAM usage now 598.0 MiB


/var/folders/38/1vcxjcxx0z75mdh8whh4pw6r0000gn/T/ipykernel_70604/3237411982.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_female[columns_to_impute] = imputer.fit_transform(df_female[columns_to_impute])


## Traitement des valeurs aberrantes

In [164]:
# A faire
# TODO : Guillaume

In [164] used 3.0 MiB RAM in 0.11s (system mean cpu 18%, single max cpu 100%), peaked 0.0 MiB above final usage, current RAM usage now 601.0 MiB


## Encodage des données

Encodage des médailles

In [165]:
# Encodage du sexe
df_complete['Sex'].replace(['F','M'],['0', '1'],inplace=True)

# Encodage des médailles 
df_complete['Medal'].replace(['Gold','Silver','Bronze'],['5', '3','1'],inplace=True)

# transform the medal into numeric values
df_complete['Medal'] = df_complete.Medal.fillna('0')

#JO Hivers = 0; été = 1
df_complete['Season'].replace(['Winter','Summer'],['0', '1'],inplace=True)
df_complete = df_complete.drop(df_complete[df_complete['Season'] == '0'].index)

# Encodage de l'âge
df_complete['Age'] = pd.cut(df_complete.Age, range(0, 80, 5), right=False)

# Encodage du poids
df_complete['Weight'] = pd.cut(df_complete.Weight, range(0, 230, 10), right=False)

# Encodage de la taille
df_complete['Height'] = pd.cut(df_complete.Height, range(0, 250, 10), right=False)

# Encodage du sport 
encoder=preprocessing.OneHotEncoder(sparse_output=False)
encoder.fit(df_complete[['Sport']])
columns_name = encoder.get_feature_names_out(['Sport'])
df_encoded = pd.DataFrame(encoder.transform(df_complete[['Sport']]))
df_encoded.columns = columns_name
df_complete= pd.concat([df_complete, df_encoded], axis=1)

df_complete = df_complete.drop('Season', axis=1)

In [165] used 102.4 MiB RAM in 0.31s (system mean cpu 58%, single max cpu 100%), peaked 0.0 MiB above final usage, current RAM usage now 703.4 MiB


In [166]:
# Export du dataframe
df_complete.to_csv('data/df_complete.csv', index=False)

In [166] used 9.6 MiB RAM in 2.66s (system mean cpu 41%, single max cpu 100%), peaked 0.0 MiB above final usage, current RAM usage now 712.9 MiB


### Attribution score et poids médaille (Code Noé )

### Nettoyage des colonnes inutiles pour l'analyse

In [167]:
# Drop les colonnes qui ne servent à R

In [167] used 0.3 MiB RAM in 0.10s (system mean cpu 0%, single max cpu 0%), peaked 0.0 MiB above final usage, current RAM usage now 713.3 MiB


## Traitement des valeurs manquantes 

Effectivement notre base de données possède de nombreuses valeurs manquantes 

In [168]:
# df.isna().sum()

In [168] used 0.0 MiB RAM in 0.11s (system mean cpu 44%, single max cpu 78%), peaked 0.0 MiB above final usage, current RAM usage now 713.3 MiB


In [169]:
### Première solution : Web Scrapping (code Nathan + FLO)

In [169] used 0.0 MiB RAM in 0.10s (system mean cpu 27%, single max cpu 56%), peaked 0.0 MiB above final usage, current RAM usage now 713.3 MiB


# Data understanding

## EDA (graph, pivot table ,Matrix confusion...)

In [170]:
# TODO: Guillaume / Noé / Flo

In [170] used 0.0 MiB RAM in 0.11s (system mean cpu 34%, single max cpu 53%), peaked 0.0 MiB above final usage, current RAM usage now 713.3 MiB


## Tests de corrélations/lois stats (Shapiro-Wilk, Spearman, Pearson)

In [171]:
# TODO: Noé

In [171] used 0.0 MiB RAM in 0.11s (system mean cpu 29%, single max cpu 67%), peaked 0.0 MiB above final usage, current RAM usage now 713.3 MiB


## Aggrégation des données par Team/Année/Event pour pouvoir calculer les médailles

In [ ]:

since20000_data_teams = pd.DataFrame(since2000_data.groupby(['Team', 'Year', 'Event', 'Medal']).mean(numeric_only=True))
